In [1]:
##Se cargan los paquetes
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import time


In [2]:
#importacion de los datos
dataset = read_csv('BDlista2018.csv', header=0, index_col=0, low_memory=False)

In [3]:
#obtener los encabezados
Encabezados=dataset.columns.values.tolist()
print(Encabezados)


['Entidad_federativa', 'Tipo_de_pesca', 'Year', 'Mes', 'Dia', 'Oficina', 'Latitud', 'Longitud', 'Folio', 'Duracion', 'Dias_Efectivos', 'Sitio_Desembarque_Clave', 'Sitio_Desembarque', 'Zona_Captura', 'Zona_Captura_Clave', 'Especie_Clave', 'Especie_nombre_general', 'Especie_nombre', 'Especie_nombre_cientifico', 'Presentacion', 'Class', 'Order', 'Family', 'Genus', 'Ganancia_total_Mxn', 'Peso_Kg', 'Precio_Mxn', 'Zona', 'Pesqueria', 'Region']


### Se clasifica cada registro por las cuatro zonas que se manejan según su oficina de arribo. 

In [4]:
#obtener la lista de las oficinas que se tienen dentro de la base de datos
Oficinas=dataset.Oficina.values.tolist()#  extrae la columna de oficinas como una lista
a=pd.Series(Oficinas, dtype="category") #combierte de lista a serie
acat=a.cat.categories #extrae las categorías de la serie
oficinas=pd.Series.tolist(acat) #convierte las categorias en una lista
frecOficina=pd.value_counts(dataset['Oficina'])

In [5]:
#Se obtiene la lista de las pesquerías 
Pesqueria=dataset.Pesqueria.values.tolist()
c=pd.Series(Pesqueria,dtype="category")
acatp=c.cat.categories
pesquerias=pd.Series.tolist(acatp)
frecPesqueria=pd.value_counts(dataset['Pesqueria'])

In [6]:
#Se obtiene la lista de los años
Anos=dataset.Year.values.tolist()
c=pd.Series(Anos,dtype="category")
acaty=c.cat.categories
años=pd.Series.tolist(acaty)
frecAnos=pd.value_counts(dataset['Year'])

In [7]:
#Se obtiene la lista de los meses
Mes=dataset.Mes.values.tolist()
c=pd.Series(Mes,dtype="category")
acatm=c.cat.categories
meses=pd.Series.tolist(acatm)

frecMes=pd.value_counts(dataset['Mes'])

In [8]:
#Se crea una lista con los valores filtrados y sumados de las capturas para cada oficina, pesquería, mes y año.
df=['Oficina','Pesqueria','año','mes','Peso_Kg','valor','Frecuencia'] #se definen los encabezados del arreglo
df1=pd.DataFrame(columns=df) 

In [9]:
contador=0
for i in tqdm.tqdm(range(0,len(oficinas))):
    V=dataset.Oficina==acat[i]
    for j in range(0,len(pesquerias)):
        VV=dataset.Pesqueria==acatp[j]
        for k in range(0,len(años)):
            VVV=dataset.Year==acaty[k]
            for l in range(0,len(meses)):
                VVVV=dataset.Mes==acatm[l]#genera lista donde ambos son verdaderos
                M=V & VV & VVV & VVVV 
                B=sum(dataset.Peso_Kg[M==True])#muestra los registros que cumplen la condición anterior
                C=sum(dataset.Ganancia_total_Mxn[M==True])#muestra la suma del valor para el año en esa oficina
                fr=dataset.Peso_Kg[M==True].size
                df1.loc[contador]=[acat[i],acatp[j],acaty[k],acatm[l],B,C,fr]
                contador=contador+1

100%|██████████| 11/11 [07:15<00:00, 39.55s/it]


In [10]:
df1.to_csv('bdmensualclasificada.csv')


In [11]:
#Se crea una columna con las fechas en formato yyyy-mm-dd y se une al arreglo anterior
dt=['DATE']
dt=pd.DataFrame(columns=dt)
contador=0
for x in range(0,len(df1)):
    z=int(df1.values[x,2:3])
    zZ=int(df1.values[x,3:4])
    zzz=datetime(z,zZ,1)
    dt.loc[contador]=[zzz.strftime("%Y-%m-%d")]
    contador=contador+1

df1 = df1.assign(Date=dt.values)


In [12]:
df3 = pd.DataFrame(df1)
df3.Oficina.unique()

array(['BAHIA_ASUNCION', 'BAHIA_TORTUGAS', 'CABO_SAN_LUCAS',
       'CD._CONSTITUCION', 'GUERRERO_NEGRO', 'LA_PAZ', 'LORETO',
       'PUERTO_ADOLFO_LOPEZ_MATEOS', 'PUNTA_ABREOJOS', 'SANTA_ROSALIA',
       'SAN_CARLOS'], dtype=object)

In [13]:
df=['Zona']
ZNc=[]
ZN=pd.DataFrame(ZNc,columns=df)

i=0

for i in range(0,len(df1)):
                if df1.Oficina[i]=='LORETO' or df1.Oficina[i]== 'LA_PAZ' or df1.Oficina[i]== 'SANTA_ROSALIA' or df1.Oficina[i]=='CABO_SAN_LUCAS':
                    nd1='zn1'
                    ZN.loc[i]=[nd1]
                    i=i+1
                elif  df1.Oficina[i]=='PUERTO_ADOLFO_LOPEZ_MATEOS' or df1.Oficina[i]=='CD._CONSTITUCION'or df1.Oficina[i]=='SAN_CARLOS':
                    nd1='zn2'
                    ZN.loc[i]=[nd1]
                    i=i+1
                elif df1.Oficina[i]=='BAHIA_TORTUGAS' or df1.Oficina[i]=='BAHIA_ASUNCION'or df1.Oficina[i]=='PUNTA_ABREOJOS':
                    nd1='zn3'
                    ZN.loc[i]=[nd1]
                    i=i+1
                elif df1.Oficina[i]=='GUERRERO_NEGRO':
                    nd1='zn4'
                    ZN.loc[i]=[nd1]
                    i=i+1
                else:
                    nd1='NaN'
                    ZN.loc[i]=[nd1]
                    i=i+1


In [20]:
(len(ZN)/len(dataset))*100 #porcentaje de resumen de la bd 

4.136845207818518

In [14]:
df1 = df1.assign(Zona=ZN.values)


In [22]:
df1

,Oficina,Pesqueria,year,mes,peso,valor,Frecuencia,Date,Zona
0,BAHIA_ASUNCION,ABULON,2001,4,12219.0,1098095.5,10,2001-04-01,zn3
1,BAHIA_ASUNCION,ABULON,2001,8,0.0,0.0,0,2001-08-01,zn3
2,BAHIA_ASUNCION,ABULON,2001,12,0.0,0.0,0,2001-12-01,zn3
3,BAHIA_ASUNCION,ABULON,2001,1,0.0,0.0,0,2001-01-01,zn3
4,BAHIA_ASUNCION,ABULON,2001,2,0.0,0.0,0,2001-02-01,zn3
...,...,...,...,...,...,...,...,...,...
33259,SAN_CARLOS,TUNIDOS,2018,3,0.0,0.0,0,2018-03-01,zn2
33260,SAN_CARLOS,TUNIDOS,2018,5,0.0,0.0,0,2018-05-01,zn2
33261,SAN_CARLOS,TUNIDOS,2018,11,0.0,0.0,0,2018-11-01,zn2
33262,SAN_CARLOS,TUNIDOS,2018,10,0.0,0.0,0,2018-10-01,zn2


In [15]:
df1.to_csv('Base2018-2.csv')